# C-Town year simulation using weeks

This notebook can be used to run a C-Town simulation for 9 months in a "week-by-week" (weeks of 10-days) basis
The notebook reads from .csv files to define the tank level initial conditions for the week and the demand patterns for that week. 
The tank initial levels are stored in "tank_initial_conditions" and the demand points are defined in "starting_demand_points" and "three_years_demands_ctown"

In [1]:
# load module
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import wntr
import wntr_utils
from sklearn.preprocessing import MinMaxScaler
from random import shuffle
import csv
import random

In [2]:
week_index = 0
total_demands = pd.read_csv('three_year_demands_ctown.csv', index_col=0)
demand_starting_points = pd.read_csv('starting_demand_points.csv', index_col=0)
initial_tank_levels = pd.read_csv('tank_initial_conditions.csv', index_col=0)
network_delays = pd.read_csv('network_links_delay.csv', index_col=0)
network_losses = pd.read_csv('network_loss_small.csv', index_col=0)

In [3]:
# load EPANET model
inp_file = 'ctown_map_with_controls.inp'
wn = wntr.network.WaterNetworkModel(inp_file)

C:\Users\Azulm\Anaconda3\lib\site-packages\wntr\epanet\io.py:2085: UserWarning: Not all curves were used in "ctown_map_with_controls.inp"; added with type None, units conversion left to user
  warnings.warn('Not all curves were used in "{}"; added with type None, units conversion left to user'.format(self.wn.name))


In [4]:
wn.options.time.duration

864000

In [5]:
wn.options.time.duration

864000

In [6]:
wn.options.time.hydraulic_timestep

300

In [7]:
week_index = 0
week_start = demand_starting_points.iloc[week_index][0]

In [9]:
week_demands = total_demands.loc[week_start:week_start+240:]

In [10]:
# Replace the demand patterns with the random patterns for this week
names, patterns =[],[]
for name, pat in wn.patterns():  
    pat.multipliers = week_demands[name].values.tolist()

In [11]:
wn.options.time.duration

864000

In [12]:
for pat in wn.patterns():  
    print(pat)

('DMA1_pat', <Pattern 'DMA1_pat', multipliers=array([0.819, 0.633, 0.634, 0.639, 0.538, 0.491, 0.565, 0.555, 0.536, 0.602, 0.567, 0.512, 0.528, 0.519, 0.384, 0.482, 0.426, 0.35 , 0.327, 0.394, 0.36 , 0.623, 0.691, 0.643, 0.709, 0.676, 0.643, 0.608, 0.566, 0.553, 0.504, 0.538, 0.558, 0.547, 0.559, 0.587, 0.539, 0.466, 0.438, 0.479, 0.486,
       0.335, 0.345, 0.337, 0.391, 0.494, 0.703, 0.693, 0.693, 0.738, 0.565, 0.641, 0.599, 0.533, 0.534, 0.536, 0.538, 0.547, 0.566, 0.599, 0.537, 0.48 , 0.468, 0.539, 0.42 , 0.341, 0.4  , 0.367, 0.405, 0.481, 0.625, 0.781, 0.731, 0.7  , 0.613, 0.596, 0.633, 0.596, 0.49 , 0.494, 0.566, 0.543,
       0.514, 0.585, 0.492, 0.502, 0.457, 0.624, 0.475, 0.325, 0.346, 0.339, 0.42 , 0.522, 0.643, 0.772, 0.727, 0.723, 0.644, 0.613, 0.575, 0.583, 0.546, 0.556, 0.6  , 0.63 , 0.553, 0.57 , 0.522, 0.465, 0.443, 0.606, 0.473, 0.354, 0.329, 0.366, 0.427, 0.504, 0.702, 0.727, 0.754, 0.723, 0.612,
       0.594, 0.554, 0.538, 0.587, 0.508, 0.606, 0.627, 0.542, 0.518, 0.

In [13]:
# Replace the initial tank levels with the values stored in the .csv file
for i in range(1,8):    
    wn.get_node('T' + str(i)).init_level = float(initial_tank_levels.iloc[week_index]['T'+ str(i)])

In [14]:
initial_tank_levels.iloc[week_index]

T1    5.622006
T2    1.373781
T3    2.109871
T4    3.680605
T5    1.466813
T6    1.915251
T7    4.015033
Name: 0, dtype: float64

In [15]:
wn.write_inpfile('ctown_year_simulation_week_0.inp')

In [16]:
# Simulate hydraulics
sim = wntr.sim.WNTRSimulator(wn,mode='PDD')

In [17]:
# this would run the simulation. 
# Mohammad, using this notebook as base you could change the notebook to run all the 38 weeks. Notice that the variable week_index 
# is the one controlling which week is being simulated
results_new = sim.run_sim()

TypeError: unsupported operand type(s) for /: 'float' and 'NoneType'

In [15]:
pressure_results = results_new.node['pressure']
pressure_results.to_csv('test_ctown_week_1_wntr.csv')